# Chương này bao gồm những kiến thức chính:

* Đếm từ và tần số xuất hiện để phân tích ngữ nghĩa
* Dự đoán xác suất xuất hiện từ với Zip'f Law
* Biểu diễn vecto từ và làm thế nào để bắt đầu sử dụng chúng
* Tìm document liên quan từ một corpus sử dụng tần số document nghịch đảo
* Ước tính sự giống nhau của document với cosine và Okapi BM25

Trong chương này, chúng ta sẽ xem xét 3 cách mạnh mẽ để biểu diễn từ và tầm quan trọng của chúng trong 1 document:
* <i>Bags of words</i>: Vector hóa số từ hoặc tần suất
* <i>Bags of n-grams</i>: Đếm số cặp từ (bigram), triplets (trigrams), ...
* <i>TF-IDF vectors</i>: Chấm điểm từ, đại diện cho mức độ quan trọng của chúng.

QUAN TRỌNG: TF-IDF là term frequenct times inverse document frequency.<br/>
Team frequencies là đếm mỗi từ trong document, cái mà bạn đã học ở chương trước. Inverse document frequency nghĩa là bạn sẽ chia mỗi số từ đó cho số từ trong tài liệu.
    

Mỗi một kỹ thuật có thể được áp dụng riêng rẽ hoặc như một phần của pipeline NLP. Đây là mô hình thống kê dựa trên frequency (tần số) xuất hiện. Phần sau của quyển sách này, bạn sẽ có cách khác để tìm hiểu sâu hơn về mối quan hệ giữa các từ và mô hình giữa chúng.

Nhưng máy học NLP "shallow" (nông cạn) là mạnh mẽ và hữu ích cho nhiều ứng dụng thực tiễn như lọc spam và sentimen analysis (phân tích cảm xúc).

## Bag of words

Ở chương trước, bạn đã tạo một mô hình không gian vecto đầu tiên của một văn bản. Bạn sử dụng one-hot encoding cho mỗi từ và rồi tổ hợp tất cả các vector với một phép toán OR (hoặc sum) để tạo một vector biểu diễn cho 1 text. Và vecto bag-of-words nhị phân này cần một lượng index lớn để biểu diễn document khi load vào trong 1 cấu trúc như là Pandas DataFrame.

Ví dụ về đếm số lần xuất hiện của từ:

In [1]:
from nltk.tokenize import TreebankWordTokenizer
sentence = """The faster Harry got to the store, the faster Harry,
... the faster, would get home."""
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(sentence.lower())

In [2]:
tokens

['the',
 'faster',
 'harry',
 'got',
 'to',
 'the',
 'store',
 ',',
 'the',
 'faster',
 'harry',
 ',',
 '...',
 'the',
 'faster',
 ',',
 'would',
 'get',
 'home',
 '.']

Với list đơn của bạn, bạn muốn lấy unique words từ document và đếm chúng. Dictionary trong python phục vụ tốt cho mục đích này, và bởi vì bạn muốn đếm số từ, bạn có thể sử dụng <b>Counter</b>, như bạn đã làm ở chương trước:

In [3]:
from collections import Counter
bag_of_words = Counter(tokens)
bag_of_words

Counter({'the': 4,
         'faster': 3,
         'harry': 2,
         'got': 1,
         'to': 1,
         'store': 1,
         ',': 3,
         '...': 1,
         'would': 1,
         'get': 1,
         'home': 1,
         '.': 1})

Như dictionary Python, thứ tự các key bị xáo trộn. Sự sắp xếp mới này tối ưu cho lưu trữ, cập nhật và phục hồi, không thích hợp hiển thị. Thông tin về thứ tự ban đầu của các từ bị loại bỏ.

NOTE: Một <b>collections.Counter</b> là một đối tượng collection không thứ tự. Hơn nữa, gọi là một bag hoặc multiset. Phụ thuộc vào nền tảng của bạn và phiên bản Python, bạn có thể thấy rằng một <b>Counter</b> được hiển thị trong một thứ tự hợp lý, giống như thứ tự từ vựng hoặc thứ tự xuất hiện trong tokens của bạn. Nhưng cũng giống như dict chuẩn của Python, bạn không thể tin vào sắp xếp của tokens (keys) trong một <b>Counter</b>

Đối với document ngắn như thế này, bag of words không thứ tự cũng bao gồm rất nhiều thông tin quan trọng về mục đích và ngữ nghĩa. Và thông tin trong bag of word là đủ để thực hiện 1 số thứ hữu ích như detect sapm, phân tích cảm xúc. Nó có thể như 1 cái túi, nhưng nó chứa đầy ý nghĩa và thông tin. Vì vậy, sắp xếp những từ này theo một trật tự dễ dàng hơn để hiểu. Đối tượng <b>Counter</b> có một phương thức thực hiện, <b>most_common</b> cho mục đích này:

In [4]:
bag_of_words.most_common(4)

[('the', 4), ('faster', 3), (',', 3), ('harry', 2)]

Mặc định, most_common() liệt kê tất cả các tokens phổ biến nhất, nhưng bạn có thể giới hạn số lượng list ra bằng cách thêm số vào như số 4 bên trên.

Đặc biệt, số lần một từ xuất hiện trong 1 document được gọi là <b>tearm frequency</b> (tần số), viết tắt là TF. <br/>
Vì vậy 4 term hoặc token đầu của bạn là  “the,” “,”, “harry,” and “faster.”. Tuy nhiên từ "the" và ký hiệu "," là không quan trọng trong ý nghĩa của document. Và những token uninfomative (không thông tin) xuất hiện nhiều. Vì vậy, trong ví dụ này, chúng ta sẽ loại bỏ chúng, cùng với một danh sách stop word và dấu chấm câu. Điều này không luôn luôn loại ở được mọi trường hợp, nhưng cho đến thời điểm này thì nó là ví dụ đơn giản. Bạn thấy rằng các từ "harry" và "faster" là các token hàng đầu ở trong vector TF của bạn (bag of word.<br/>
Nào, hãy tính toán term frequency của "harry" từ đối tượng <b>Counter</b> bạn định nghĩa ở trên:

In [5]:
times_harry_appears=bag_of_words["harry"] # số lần xuất hiện của từ harry
print(times_harry_appears)
num_unique_words = len(bag_of_words) # số unique token nguồn
print(num_unique_words)
tf = times_harry_appears/num_unique_words
print(round(tf,4))

2
12
0.1667


Hãy dừng lại vài giây và tìm hiểu sâu hơn một chút về team frequency, một từ mà chúng ta sử dụng thường xuyên ở cuốn sách này.<br/>
Khi nói bạn đã tìm được từ "dog" 3 lần ở document A và 100 lần ở document B. Rõ ràng "dog" là quan trọng hơn trong document B. Nhưng khoan. Hãy nói bạn tìm ở document A là một email 30 từ và document B là tiểu thuyết Chiến tranh và hòa bình (War and Peace) (xấp xỉ 580,000 từ). Phân tích như sau:<br/>

$$TF("dog",document_A)=3/30=.1$$
$$TF("dog",document_B)=100/580000=0.0017$$

Bây giờ bạn có thể thấy rằng các phân tích mô tả một cái gì đó về 2 văn bản và mối quan hệ giữa chúng với từ "dog". Vì vậy, thay vì đếm số từ thô để mô tả document của bạn trong một corpus, bạn có thể chuẩn hóa với term frequencties. Cũng như thế, bạn có thể tính toán mỗi từ và lấy được độ quan trọng tương đối của từ đó với document.Bạn đã thực hiện một số tiến bộ lớn trong việc số hóa văn bản, vượt ra ngoài chỉ sự hiện diện hay vắng mặt của một từ.

Loại bỏ stop word:

In [6]:
import nltk
nltk.download('stopwords', quiet=True)

True

In [7]:
tokens

['the',
 'faster',
 'harry',
 'got',
 'to',
 'the',
 'store',
 ',',
 'the',
 'faster',
 'harry',
 ',',
 '...',
 'the',
 'faster',
 ',',
 'would',
 'get',
 'home',
 '.']

In [8]:
stopwords = nltk.corpus.stopwords.words('english')
tokens = [x for x in tokens if x not in stopwords]
kite_counts = Counter(tokens)
kite_counts

Counter({'faster': 3,
         'harry': 2,
         'got': 1,
         'store': 1,
         ',': 3,
         '...': 1,
         'would': 1,
         'get': 1,
         'home': 1,
         '.': 1})

In [9]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## Vectorizing

Bạn đã có thể chuyển đổi văn bản của bạn thành số ở mức cơ bản. Nhưng bạn vẫn chỉ lưu trữ chúng ở dictionary, tiếp theo chúng ta sẽ tạo một vector của số từ.

In [10]:
tokens

['faster',
 'harry',
 'got',
 'store',
 ',',
 'faster',
 'harry',
 ',',
 '...',
 'faster',
 ',',
 'would',
 'get',
 'home',
 '.']

In [12]:
document_vector = []
doc_length = len(tokens)
for key, value in kite_counts.most_common():
    document_vector.append(value/doc_length)

In [13]:
document_vector

[0.2,
 0.2,
 0.13333333333333333,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667]

Có thể tăng tốc xử lý bằng numpy

colections của các từ (bộ sưu tập các từ) trong vocabulary được gọi là <i>lexicon</i>

In [14]:
docs = ["The faster Harry got to the store, the faster and faster Harry would get home."]
docs.append("Harry is hairy and faster than Jill.")
docs.append("Jill is not as hairy as Harry.")

In [15]:
docs

['The faster Harry got to the store, the faster and faster Harry would get home.',
 'Harry is hairy and faster than Jill.',
 'Jill is not as hairy as Harry.']

In [16]:
doc_tokens = []
for doc in docs:
    doc_tokens += [sorted(tokenizer.tokenize(doc.lower()))]

In [17]:
len(doc_tokens[0])

17

In [18]:
doc_tokens

[[',',
  '.',
  'and',
  'faster',
  'faster',
  'faster',
  'get',
  'got',
  'harry',
  'harry',
  'home',
  'store',
  'the',
  'the',
  'the',
  'to',
  'would'],
 ['.', 'and', 'faster', 'hairy', 'harry', 'is', 'jill', 'than'],
 ['.', 'as', 'as', 'hairy', 'harry', 'is', 'jill', 'not']]

In [19]:
all_doc_tokens = sum(doc_tokens, [])

In [20]:
len(all_doc_tokens)

33

In [21]:
all_doc_tokens

[',',
 '.',
 'and',
 'faster',
 'faster',
 'faster',
 'get',
 'got',
 'harry',
 'harry',
 'home',
 'store',
 'the',
 'the',
 'the',
 'to',
 'would',
 '.',
 'and',
 'faster',
 'hairy',
 'harry',
 'is',
 'jill',
 'than',
 '.',
 'as',
 'as',
 'hairy',
 'harry',
 'is',
 'jill',
 'not']

In [22]:
lexicon = sorted(set(all_doc_tokens))

In [23]:
lexicon

[',',
 '.',
 'and',
 'as',
 'faster',
 'get',
 'got',
 'hairy',
 'harry',
 'home',
 'is',
 'jill',
 'not',
 'store',
 'than',
 'the',
 'to',
 'would']

In [24]:
len(lexicon)

18